# 1 Linear Regression - curve fitting (Python)

## 1.1 Data generation

\begin{equation}
t = w_2x^2 + w_1x +w_0
\end{equation}


This is the "true" model for the curve.  
First we sample $n$ equidistant values for $x$, approxaimtely in the range of $[0,1]$; Then compute $n$ corresponding target values for $t$ using the above polynomial equation; To "simulate" the real-world scenario where the observed data usually come with noise, add random Gaussian noise to our target values.

In [ ]:
import random

# get ground-truth data from the "true" model 
n = 20  # number of data samples
x = [(idx-round(n/2))/(n/2) for idx in range(n)]
print(x)

w = [1, 2, 3]
t = [xn**2*w[2] + xn*w[1] + w[0] for xn in x]
print(t)

# "simulated" observed target values
std_noise = 0.2
t_observed = [t[idx]+random.gauss(0,std_noise) for idx in range(n)]

We use python plotting library [matplotlib](https://matplotlib.org/) to plot the red curve (directly from the underlying model) and the observed blue data points.

In [ ]:
# this is useful for jupyter notebook only
%matplotlib inline


import matplotlib.pyplot as plt

# plot the curve and the noise-corrupted data
plt.plot(x,t,'r')
plt.plot(x,t_observed,'bo')

## 1.2 Model Fitting
For numerical computations such as curve fitting, we can use the powerful [numpy](http://www.numpy.org/). In this tutorial, we use [*numpy.linalg.lstsq*](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.lstsq.html#numpy.linalg.lstsq) to solve [a system of linear equations](https://en.wikipedia.org/wiki/System_of_linear_equations) $\textbf{a}\textbf{x}=\textbf{b}$ for a [least sqaures solution](https://en.wikipedia.org/wiki/Least_squares).

Re-arrange to a linear system in matrix form $\textbf{a}\textbf{x}=\textbf{b}$ (N.B. the x is the unknown here, not the learning input x):

\begin{equation}
\textbf{t} = \textbf{X}\textbf{w}
\end{equation}

that is:

\begin{equation}
\begin{vmatrix}
t_{(1)} \\ t_{(2)} \\ \cdots \\ t_{(n)}
\end{vmatrix} = \begin{vmatrix}
x^2_{(1)} & x_{(1)} & 1 \\
x^2_{(2)} & x_{(2)} & 1 \\
& \cdots \\
x^2_{(n)} & x_{(n)} & 1 
\end{vmatrix} \cdot
\begin{vmatrix}
w_2 \\ w_1 \\ w_0
\end{vmatrix}
\end{equation}



In [ ]:
import numpy as np

# then use nunmpy for a least-square solution to the linear system "Xw=t"
t_observed = np.reshape(t_observed, [-1, 1])
x_1 = np.reshape(x, [-1, 1])
x_2 = np.square(x_1)
x_0 = np.ones_like(x_1)
X = np.concatenate([x_2,x_1,x_0],1)
# print to check the inputs
print(X)
print(X.shape)
print(t_observed.shape)
w_estimate = np.linalg.lstsq(X, t_observed, rcond=None)
print(w_estimate[0])  # print the output

# plot to see the estimated curve, i.e.
# t_estimate = [xn**2*w_estimate[2]+xn*w_estimate[1]+w_estimate[0] for xn in x]
# but matrix multiplication is more compact:
t_estimate = np.matmul(X,w_estimate[0])

*w_estimate* should be the best estimate of the weights $w$.  
Finally, we can add the green curve to the previous plot to visualise the estimated polynomial.

In [ ]:
plt.plot(x,t,'r')
plt.plot(x,t_observed,'bo')
plt.plot(x,t_estimate,'g')

## 1.3 Model Fitting Error
From the above plot, we can see that the green curve (i.e. the estimated polynomial model) does not go through all the observed points (in blue) which were used in fitting the model. This is also known as training error. There is also discrepancy between underlying curve (in red) and the estimated one (in green), but this difference may not be possible to calculate without access to the true underlying model. 

In [ ]:
# residuals:
Residuals = t_estimate-t_observed
SR = np.sum(np.square(Residuals))  # sums of residuals: b - a*x
# root-mean-square error
RMSE = np.sqrt(np.mean(np.square(Residuals)))
print(SR)
print(RMSE)

# plot the error distribution
plt.hist(Residuals)

## Questions
### Sample size
- Effect on changing the sample size on model fitting and its errors.
- How many samples are needed?

### Model fitting
- What is the objective (loss) function?
- What is the difference to the true target values?
- How to measure goodness-of-fit?